<a href="https://colab.research.google.com/github/leonaardoas/Demand-Forecast-/blob/main/Previs%C3%A3o_de_Demanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error as mae

In [2]:
read = pd.read_csv('/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados/relatorio_01_2020.csv', sep =";", thousands =".", decimal = ",")
read

,Produto,,Código,Localização,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda
0,Camisa Manga Curta - Castelo Novo Tamanho:6 a...,,MC-CASTELO6,Estoque,95.0,32.0,45.00,4275.00,1440.00,0.0,0.00,0.0,4275.00
1,Camisa Manga Curta - Castelo Novo Tamanho:10 ...,,MC-CASTELO10,Estoque,89.0,39.0,47.50,4227.50,1852.50,0.0,0.00,0.0,4227.50
2,Camisa Manga Curta - Castelo Novo Tamanho:8 a...,,MC-CASTELO8,Estoque,74.0,27.0,47.50,3515.00,1282.50,0.0,0.00,0.0,3515.00
3,Camisa Manga Curta - Castelo Novo Tamanho:12 ...,,MC-CASTELO12,Estoque,64.0,32.0,47.50,3040.00,1520.00,0.0,4.75,0.0,3035.25
4,Camisa Manga Curta - Castelo Novo Tamanho:14 ...,,MC-CASTELO14,Estoque,51.0,21.0,50.00,2550.00,1050.00,0.0,0.00,0.0,2550.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Jaqueta Fundamental - Maple Bear Tamanho:16 an...,,JMICF-16,NaN,1.0,0.0,123.80,123.80,0.00,0.0,0.00,0.0,123.80
358,Camisa Pólo - Colégio Bruno Ostmann Tamanho:P,,pol-cbop,Estoque,1.0,0.0,55.00,55.00,0.00,0.0,0.00,0.0,55.00
359,Short Saia Microfibra 100% Poliéster Castelo E...,,ss-castinf4,NaN,1.0,1.0,40.95,40.95,40.95,0.0,0.00,0.0,40.95
360,Camiseta Regata Branca - Colégio Aprovado Tama...,,regb-apv16,Estoque,1.0,1.0,47.00,47.00,47.00,0.0,0.00,0.0,47.00


In [3]:

def carregar_dados(arquivo):
  data = pd.read_csv(f"/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados/{arquivo}", sep=";")
  filename = arquivo.replace(".csv", "")
  filename = filename.split("_")
  month = filename[1]
  year = filename[2]
  data.insert(0, "day", 1) #posição, nome da coluna, elemento a ser inserido
  data.insert(0, "month", month)
  data.insert(0, "year", year)
  data["date"] = pd.to_datetime(data[["year", "month", "day"]])
  data.drop(["day", "month", "year"], axis=1, inplace=True)

  return data

In [ ]:
sheets = []

for arquivo in os.listdir("/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados"):
  func = carregar_dados(arquivo)
  sheets.append(func)

dt = pd.concat(sheets, ignore_index=True)
dt


In [ ]:
totais = dt[dt.Produto == 'Totais']
totais = len(totais)
totais

df = dt[dt.Produto != 'Totais'].reset_index(drop=True)
df
df.info()

null = df.isnull().sum()
null

In [6]:
df.drop("Localização", inplace=True, axis=1)
df.drop(df[df["Código"].isnull()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
modelo = df["Produto"].apply(lambda x: x.split("-")[0])
escolaetamanho = df["Produto"].apply(lambda x: x.split("-")[-1])
escolaetamanho

escola = escolaetamanho.apply(lambda x: x.split("Tamanho:")[0])
tamanho = escolaetamanho.apply(lambda x: x.split("Tamanho:")[-1])
tamanho

df.insert(0, "Escola", escola)
df.insert(1, "Modelo", modelo)
df.insert(2, "Tamanho", tamanho)
df.drop("Produto", axis=1, inplace=True)
df


In [ ]:
escolas = df["Escola"].value_counts()
escolas

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import process

In [ ]:
def padronizar_nomes(df, coluna, referencia, limite=80):

    # Cria um dicionário de mapeamento de nomes
    escolapadronizadas = {}

    for nome in df[coluna].unique():
        melhor_correspondencia = process.extractOne(nome, referencia, score_cutoff=limite)
        if melhor_correspondencia:
            melhor_nome, _ = melhor_correspondencia
            escolapadronizadas[nome] = melhor_nome

    # Atualiza a coluna do DataFrame com os nomes padronizados
    df[coluna] = df[coluna].map(escolapadronizadas).fillna(df[coluna])

    return df

# Lista de nomes de referência padronizados
referencia = [
    'Maple Bear',
    'CE Carvalho',
    ' Colégio Castelo',
    'Colégio Aprovado',
    'Colégio Ativo',
    'Colégio Bruno Ostmann',
    'Colégio Radan',
    'Creche Escola Arte de Crescer',
    'Creche Escola Primeiros Passos',
    'Borracha Lapis',
]

# Aplicar a função ao DataFrame
df = padronizar_nomes(df, 'Escola', referencia)

df


In [ ]:
xx = df["Escola"].value_counts()
xx

In [ ]:
df['Escola'] = df['Escola'].str.strip()
df['Escola'] = df['Escola'].apply(lambda x: "Colégio Castelo" if x == "Castelo Novo" else x)
xp = df["Escola"].value_counts()
xp

In [45]:
borracha = df[
    (df["Escola"] == "Borracha Lapis") |
    (df["Escola"] == "Camisa Manga Curta Fundamental I") |
    (df["Escola"] == "Camisa Manga Curta Fundamental lI") |
    (df["Escola"] == "Modelo Escolar")
]

borracha

# Não foi possível identificar o nome das escolas nem pelo código, portanto optei por remover
remover = ["Borracha Lapis",
           "Camisa Manga Curta Fundamental I",
           "Camisa Manga Curta Fundamental lI",
           "Modelo Escolar"
           ]

df = df.drop(df[df["Escola"].isin(remover)].index)
df.reset_index(drop=True, inplace=True)

df


,Escola,Modelo,Tamanho,,Código,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Colégio Castelo,Camisa Manga Curta,6 anos,,MC-CASTELO6,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Colégio Castelo,Camisa Manga Curta,10 anos,,MC-CASTELO10,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Colégio Castelo,Camisa Manga Curta,8 anos,,MC-CASTELO8,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Colégio Castelo,Camisa Manga Curta,12 anos,,MC-CASTELO12,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Colégio Castelo,Camisa Manga Curta,14 anos,,MC-CASTELO14,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,Colégio Ativo,Camiseta Regata Amarela,4 anos,,reg-am-ativo4,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4097,CE Carvalho,Camiseta Regata Infantil,6 anos,,regi-car6,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4098,Colégio Aprovado,Camiseta Regata Marinho,8 anos,,regm-apv8,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4099,Maple Bear,Jaqueta Infantil,4 anos,,JMICI-4,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01


In [53]:
modelosunicos= df["Modelo"].value_counts().sort_values(ascending=False)
modelosunicos[0:30]

,count
Modelo,
Camisa Manga Curta,377
Bermuda,303
Calça Legging,231
Camiseta Regata,198
Camisa Manga Curta,189
Short Saia,158
Camisa Manga Curta Raglan,134
Camisa Manga Curta Infantil,129
Camisa Manga Curta Feminino,112
